In [1]:
import luigi
import pandas as pd
import numpy as np
import os
import pickle
list_sum = sum

from mars_gym.data.task import BasePrepareDataFrames, BasePySparkTask
from mars_gym.data.utils import DownloadDataset
from mars_gym.data.dataset import (
    InteractionsDataset,
    InteractionsWithNegativeItemGenerationDataset,
)
import random
from typing import Tuple, List, Union, Callable, Optional, Set, Dict, Any
from mars_gym.meta_config import *
from pyspark.ml.feature import StandardScaler

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import collect_set, collect_list, lit, sum, udf, concat_ws, col, count, abs, date_format, \
    from_utc_timestamp, expr, min, max, mean, stddev
from pyspark.sql.functions import col, udf, size
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import explode, posexplode
from torchnlp.encoders.text.static_tokenizer_encoder import StaticTokenizerEncoder
import re
from unidecode import unidecode

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import  collect_list, lit, sum, udf, col, count, abs, max, lag, unix_timestamp
from pyspark.sql.functions import posexplode
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, FloatType
from pyspark.sql.functions import udf, struct
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import CountVectorizer as CountVectorizerSpark
from pyspark.ml.linalg import Vectors
from pyspark.sql.window import Window
from pyspark.sql.functions import when
from pyspark.ml.feature import QuantileDiscretizer

from itertools import chain
from datetime import datetime, timedelta
from tqdm import tqdm
tqdm.pandas()

In [2]:
spark

In [3]:
path = '../output/mercado_livre/dataset/session_test_dataset__processed.csv'

df      = spark.read.option("delimiter", ",").csv(path, header=True, inferSchema=True)
df      = df.withColumnRenamed("session_id", "SessionID")\
            .withColumnRenamed("event_timestamp", "Timestamp")\
            .withColumnRenamed("event_view", "ItemID")\
            .withColumn("ItemID", col("ItemID").cast("int"))\
            .withColumn("Timestamp", col("Timestamp").cast("timestamp"))\
            .orderBy(col('Timestamp'), col('SessionID'))\
            .select("SessionID", "ItemID", "Timestamp", 
                    "event_type_idx", "event_search", "domain_count", "item_id_count").limit(1000).cache()#.sample(fraction=0.01)#.limit(1000)


In [4]:
df.printSchema()

root
 |-- SessionID: long (nullable = true)
 |-- ItemID: integer (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- event_type_idx: integer (nullable = true)
 |-- event_search: string (nullable = true)
 |-- domain_count: integer (nullable = true)
 |-- item_id_count: double (nullable = true)



In [22]:
df.show(20)

+-----------+-------+--------------------+--------------+--------------------+------------+-------------+--------------------+--------------------+
|  SessionID| ItemID|           Timestamp|event_type_idx|        event_search|domain_count|item_id_count|   last_event_search|  last_event_search2|
+-----------+-------+--------------------+--------------+--------------------+------------+-------------+--------------------+--------------------+
|42949683675|     -1|2019-09-24 09:41:...|             3|        [1059, 1037]|           0|         null|                null|                null|
|42949683675|  41782|2019-09-24 09:42:...|             4|             [18752]|           0|        106.0|        [1059, 1037]|        [1059, 1037]|
|42949683675|     -1|2019-09-24 09:43:...|             3|[3912, 1059, 1037...|           0|         null|             [18752]|        [1059, 1037]|
|42949683675|     -1|2019-09-24 09:44:...|             3|  [1059, 1037, 5475]|           0|         null|[3912, 

## Lag Search

In [23]:
df = df.filter(col("SessionID") == 42949683675)
df.show(10)

+-----------+-------+--------------------+--------------+--------------------+------------+-------------+--------------------+--------------------+
|  SessionID| ItemID|           Timestamp|event_type_idx|        event_search|domain_count|item_id_count|   last_event_search|  last_event_search2|
+-----------+-------+--------------------+--------------+--------------------+------------+-------------+--------------------+--------------------+
|42949683675|     -1|2019-09-24 09:41:...|             3|        [1059, 1037]|           0|         null|                null|                null|
|42949683675|  41782|2019-09-24 09:42:...|             4|             [18752]|           0|        106.0|        [1059, 1037]|        [1059, 1037]|
|42949683675|     -1|2019-09-24 09:43:...|             3|[3912, 1059, 1037...|           0|         null|             [18752]|        [1059, 1037]|
|42949683675|     -1|2019-09-24 09:44:...|             3|  [1059, 1037, 5475]|           0|         null|[3912, 

In [24]:
df = df.dropDuplicates(['SessionID', 'ItemID', 'event_type_idx', 'event_search']).cache()
df.show(10)

+-----------+-------+--------------------+--------------+--------------------+------------+-------------+--------------------+--------------------+
|  SessionID| ItemID|           Timestamp|event_type_idx|        event_search|domain_count|item_id_count|   last_event_search|  last_event_search2|
+-----------+-------+--------------------+--------------+--------------------+------------+-------------+--------------------+--------------------+
|42949683675|     -1|2019-09-24 09:44:...|             3|  [1059, 1037, 5475]|           0|         null|[3912, 1059, 1037...|[3912, 1059, 1037...|
|42949683675|     -1|2019-09-24 09:41:...|             3|        [1059, 1037]|           0|         null|                null|                null|
|42949683675|     -1|2019-09-24 09:46:...|             3|         [1059, 977]|           0|         null|             [18752]|[3912, 1059, 1037...|
|42949683675|     -1|2019-09-24 09:43:...|             3|[3912, 1059, 1037...|           0|         null|       

In [27]:
w  = Window.partitionBy('SessionID').orderBy('Timestamp')#.rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)


In [28]:
df = df.withColumn("last_event_search",   lag(df.event_search, 1).over(w) )
df.select("ItemID", "Timestamp", "event_type_idx", "event_search", "last_event_search").show()

+-------+--------------------+--------------+--------------------+--------------------+
| ItemID|           Timestamp|event_type_idx|        event_search|   last_event_search|
+-------+--------------------+--------------+--------------------+--------------------+
|     -1|2019-09-24 09:41:...|             3|        [1059, 1037]|                null|
|  41782|2019-09-24 09:42:...|             4|             [18752]|        [1059, 1037]|
|     -1|2019-09-24 09:43:...|             3|[3912, 1059, 1037...|             [18752]|
|     -1|2019-09-24 09:44:...|             3|  [1059, 1037, 5475]|[3912, 1059, 1037...|
|1744246|2019-09-24 09:44:...|             4|             [18752]|  [1059, 1037, 5475]|
|     -1|2019-09-24 09:46:...|             3|         [1059, 977]|             [18752]|
|2084331|2019-09-24 09:47:...|             4|             [18752]|         [1059, 977]|
+-------+--------------------+--------------+--------------------+--------------------+



In [53]:
w2  = Window.partitionBy('SessionID').orderBy('Timestamp').rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)

df  = df.withColumn("last_event_search", when(col("event_type_idx") == lit(3), col("event_search")))
df  = df.withColumn("last_event_search", F.last("last_event_search", True).over(w))
df  = df.withColumn("last_event_search", F.lag("last_event_search").over(w))

df.select("ItemID", "Timestamp", "event_type_idx", "event_search", "last_event_search").show()


+-------+--------------------+--------------+--------------------+--------------------+
| ItemID|           Timestamp|event_type_idx|        event_search|   last_event_search|
+-------+--------------------+--------------+--------------------+--------------------+
|     -1|2019-09-24 09:41:...|             3|        [1059, 1037]|                null|
|  41782|2019-09-24 09:42:...|             4|             [18752]|        [1059, 1037]|
|     -1|2019-09-24 09:43:...|             3|[3912, 1059, 1037...|        [1059, 1037]|
|     -1|2019-09-24 09:44:...|             3|  [1059, 1037, 5475]|[3912, 1059, 1037...|
|1744246|2019-09-24 09:44:...|             4|             [18752]|  [1059, 1037, 5475]|
|     -1|2019-09-24 09:46:...|             3|         [1059, 977]|  [1059, 1037, 5475]|
|2084331|2019-09-24 09:47:...|             4|             [18752]|         [1059, 977]|
+-------+--------------------+--------------+--------------------+--------------------+



In [38]:
from pyspark.sql.functions import expr

df.withColumn(
    'lag', 
    expr('max(case when event_type_idx == 3 then event_search end) over (order by Timestamp rows between unbounded preceding and 1 preceding)')
  ).select("ItemID", "Timestamp", "event_type_idx", "event_search", "last_event_search", "lag").show()

+-------+--------------------+--------------+--------------------+--------------------+--------------------+
| ItemID|           Timestamp|event_type_idx|        event_search|   last_event_search|                 lag|
+-------+--------------------+--------------+--------------------+--------------------+--------------------+
|     -1|2019-09-24 09:41:...|             3|        [1059, 1037]|                null|                null|
|  41782|2019-09-24 09:42:...|             4|             [18752]|        [1059, 1037]|        [1059, 1037]|
|     -1|2019-09-24 09:43:...|             3|[3912, 1059, 1037...|             [18752]|        [1059, 1037]|
|     -1|2019-09-24 09:44:...|             3|  [1059, 1037, 5475]|[3912, 1059, 1037...|[3912, 1059, 1037...|
|1744246|2019-09-24 09:44:...|             4|             [18752]|  [1059, 1037, 5475]|[3912, 1059, 1037...|
|     -1|2019-09-24 09:46:...|             3|         [1059, 977]|             [18752]|[3912, 1059, 1037...|
|2084331|2019-09-24

## Stat Price

In [43]:
def stat_float_hist(func):
    def _stat_float_hist(arr, pad=0.0):
        arr = np.array(eval(arr))
        return float(func(arr[arr != pad]))
    return udf(_stat_float_hist, FloatType())

udf_stat_float_hist = stat_float_hist(np.sum)

_df = df.withColumn('sum_price_history',  stat_float_hist(np.sum)(col("price_history")))
_df = _df.withColumn('mean_price_history', stat_float_hist(np.mean)(col("price_history")))
_df = _df.withColumn('min_price_history',  stat_float_hist(np.min)(col("price_history")))
_df = _df.withColumn('max_price_history',  stat_float_hist(np.max)(col("price_history")))

_df = _df.toPandas().head()
_df


,SessionID,ItemID,Timestamp,event_type,last_ItemID,last_event_search,last_event_type,event_type_history,ItemID_history,timestamp_history,category_id_history,domain_id_history,price_history,sum_price_history,mean_price_history,min_price_history,max_price_history
0,0,1615991,2019-10-19 12:27:26.879,view,-1.0,"[18, 5, 12, 15, 7, 9, 15, 0, 19, 13, 1, 18, 20...",search,"['search', 'view', '0', '0', '0', '0', '0', '0...","[-1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[419846, 419845, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['-1', 'MLB135384', '0', '0', '0', '0', '0', '...","['-1', 'MLB-SMARTWATCHES', '0', '0', '0', '0',...","[-0.006754247471690178, -0.006377095822244883,...",-0.013131,-0.006566,-0.006754,-0.006377
1,0,1748830,2019-10-20 20:29:11.646,buy,1615991.0,"[14, 1, 14]",view,"['view', 'search', 'view', '0', '0', '0', '0',...","[1615991, -1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0,...","[419847, 419846, 419845, 0, 0, 0, 0, 0, 0, 0, ...","['MLB135384', '-1', 'MLB135384', '0', '0', '0'...","['MLB-SMARTWATCHES', '-1', 'MLB-SMARTWATCHES',...","[-0.006503703538328409, -0.006754247471690178,...",-0.019635,-0.006545,-0.006754,-0.006377
2,1,228737,2019-10-07 19:56:43.678,view,-1.0,"[4, 5, 19, 13, 1, 13, 1, 4, 5, 9, 18, 1, 0, 5,...",search,"['search', '0', '0', '0', '0', '0', '0', '0', ...","[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","[-0.006754247471690178, 0.0, 0.0, 0.0, 0.0, 0....",-0.006754,-0.006754,-0.006754,-0.006754
3,1,228737,2019-10-07 20:46:48.382,buy,228737.0,"[14, 1, 14]",view,"['view', 'search', '0', '0', '0', '0', '0', '0...","[228737, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[403016, 402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['MLB264021', '-1', '0', '0', '0', '0', '0', '...","['MLB-MILK_EXTRACTORS', '-1', '0', '0', '0', '...","[-0.006223048083484173, -0.006754247471690178,...",-0.012977,-0.006489,-0.006754,-0.006223
4,10000,1910400,2019-10-21 18:22:26.549,buy,1431639.0,"[14, 1, 14]",view,"['view', '0', '0', '0', '0', '0', '0', '0', '0...","[1431639, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[423081, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['MLB278239', '0', '0', '0', '0', '0', '0', '0...","['MLB-FLEA_AND_TICK_TREATMENTS', '0', '0', '0'...","[-0.006220533512532711, 0.0, 0.0, 0.0, 0.0, 0....",-0.006221,-0.006221,-0.006221,-0.006221


In [40]:
arr = np.array(eval(_df.iloc[2].price_history))

arr[arr != 0.0]

array([-0.00675425])

In [47]:
a = _df[['price_history', 'sum_price_history', 'mean_price_history', 'min_price_history', 'max_price_history']].iloc[1]
a['price_history'] = eval(a['price_history'])

In [48]:
a.price_history

price_history         [-0.006503703538328409, -0.006754247471690178,...
sum_price_history                                             -0.019635
mean_price_history                                          -0.00654502
min_price_history                                           -0.00675425
max_price_history                                            -0.0063771
Name: 1, dtype: object

In [54]:
np.array(a.price_history).max()

0.0

## Moda Array

df

In [62]:
df.toPandas().head()

,SessionID,ItemID,Timestamp,event_type,last_ItemID,last_event_search,last_event_type,event_type_history,ItemID_history,timestamp_history,category_id_history,domain_id_history,price_history
0,0,1615991,2019-10-19 12:27:26.879,view,-1.0,"[18, 5, 12, 15, 7, 9, 15, 0, 19, 13, 1, 18, 20...",search,"['search', 'view', '0', '0', '0', '0', '0', '0...","[-1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[419846, 419845, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['-1', 'MLB135384', '0', '0', '0', '0', '0', '...","['-1', 'MLB-SMARTWATCHES', '0', '0', '0', '0',...","[-0.006754247471690178, -0.006377095822244883,..."
1,0,1748830,2019-10-20 20:29:11.646,buy,1615991.0,"[14, 1, 14]",view,"['view', 'search', 'view', '0', '0', '0', '0',...","[1615991, -1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0,...","[419847, 419846, 419845, 0, 0, 0, 0, 0, 0, 0, ...","['MLB135384', '-1', 'MLB135384', '0', '0', '0'...","['MLB-SMARTWATCHES', '-1', 'MLB-SMARTWATCHES',...","[-0.006503703538328409, -0.006754247471690178,..."
2,1,228737,2019-10-07 19:56:43.678,view,-1.0,"[4, 5, 19, 13, 1, 13, 1, 4, 5, 9, 18, 1, 0, 5,...",search,"['search', '0', '0', '0', '0', '0', '0', '0', ...","[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","[-0.006754247471690178, 0.0, 0.0, 0.0, 0.0, 0...."
3,1,228737,2019-10-07 20:46:48.382,buy,228737.0,"[14, 1, 14]",view,"['view', 'search', '0', '0', '0', '0', '0', '0...","[228737, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[403016, 402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['MLB264021', '-1', '0', '0', '0', '0', '0', '...","['MLB-MILK_EXTRACTORS', '-1', '0', '0', '0', '...","[-0.006223048083484173, -0.006754247471690178,..."
4,10000,1910400,2019-10-21 18:22:26.549,buy,1431639.0,"[14, 1, 14]",view,"['view', '0', '0', '0', '0', '0', '0', '0', '0...","[1431639, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[423081, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['MLB278239', '0', '0', '0', '0', '0', '0', '0...","['MLB-FLEA_AND_TICK_TREATMENTS', '0', '0', '0'...","[-0.006220533512532711, 0.0, 0.0, 0.0, 0.0, 0...."


In [86]:
from scipy import stats

def moda_hist(arr, pad='0'):
    arr = np.array(eval(arr))
    return str(stats.mode(arr[arr != pad])[0][0])
               

udf_moda_hist = udf(moda_hist, StringType())


In [99]:
from scipy import stats
lst = "['a','b','b','a', 'c']"
moda_hist(lst)

'a'

In [88]:
_df = df.withColumn('mode_event_type_history',  udf_moda_hist(col("event_type_history")))

_df = _df.toPandas().head()
_df


,SessionID,ItemID,Timestamp,event_type,last_ItemID,last_event_search,last_event_type,event_type_history,ItemID_history,timestamp_history,category_id_history,domain_id_history,price_history,mode_event_type_history
0,0,1615991,2019-10-19 12:27:26.879,view,-1.0,"[18, 5, 12, 15, 7, 9, 15, 0, 19, 13, 1, 18, 20...",search,"['search', 'view', '0', '0', '0', '0', '0', '0...","[-1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[419846, 419845, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['-1', 'MLB135384', '0', '0', '0', '0', '0', '...","['-1', 'MLB-SMARTWATCHES', '0', '0', '0', '0',...","[-0.006754247471690178, -0.006377095822244883,...",search
1,0,1748830,2019-10-20 20:29:11.646,buy,1615991.0,"[14, 1, 14]",view,"['view', 'search', 'view', '0', '0', '0', '0',...","[1615991, -1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0,...","[419847, 419846, 419845, 0, 0, 0, 0, 0, 0, 0, ...","['MLB135384', '-1', 'MLB135384', '0', '0', '0'...","['MLB-SMARTWATCHES', '-1', 'MLB-SMARTWATCHES',...","[-0.006503703538328409, -0.006754247471690178,...",view
2,1,228737,2019-10-07 19:56:43.678,view,-1.0,"[4, 5, 19, 13, 1, 13, 1, 4, 5, 9, 18, 1, 0, 5,...",search,"['search', '0', '0', '0', '0', '0', '0', '0', ...","[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","[-0.006754247471690178, 0.0, 0.0, 0.0, 0.0, 0....",search
3,1,228737,2019-10-07 20:46:48.382,buy,228737.0,"[14, 1, 14]",view,"['view', 'search', '0', '0', '0', '0', '0', '0...","[228737, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[403016, 402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['MLB264021', '-1', '0', '0', '0', '0', '0', '...","['MLB-MILK_EXTRACTORS', '-1', '0', '0', '0', '...","[-0.006223048083484173, -0.006754247471690178,...",search
4,10000,1910400,2019-10-21 18:22:26.549,buy,1431639.0,"[14, 1, 14]",view,"['view', '0', '0', '0', '0', '0', '0', '0', '0...","[1431639, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[423081, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['MLB278239', '0', '0', '0', '0', '0', '0', '0...","['MLB-FLEA_AND_TICK_TREATMENTS', '0', '0', '0'...","[-0.006220533512532711, 0.0, 0.0, 0.0, 0.0, 0....",view


In [96]:
from scipy import stats

def moda_count_hist(arr, pad='0'):
    arr = np.array(eval(arr))
    return int(stats.mode(arr[arr != pad])[1][0])
               

udf_moda_count_hist = udf(moda_count_hist, IntegerType())


In [97]:
_df = df.withColumn('mode_count_event_type_history',  udf_moda_count_hist(col("event_type_history")))

_df = _df.toPandas().head()
_df


,SessionID,ItemID,Timestamp,event_type,last_ItemID,last_event_search,last_event_type,event_type_history,ItemID_history,timestamp_history,category_id_history,domain_id_history,price_history,mode_count_event_type_history
0,0,1615991,2019-10-19 12:27:26.879,view,-1.0,"[18, 5, 12, 15, 7, 9, 15, 0, 19, 13, 1, 18, 20...",search,"['search', 'view', '0', '0', '0', '0', '0', '0...","[-1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[419846, 419845, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['-1', 'MLB135384', '0', '0', '0', '0', '0', '...","['-1', 'MLB-SMARTWATCHES', '0', '0', '0', '0',...","[-0.006754247471690178, -0.006377095822244883,...",1
1,0,1748830,2019-10-20 20:29:11.646,buy,1615991.0,"[14, 1, 14]",view,"['view', 'search', 'view', '0', '0', '0', '0',...","[1615991, -1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0,...","[419847, 419846, 419845, 0, 0, 0, 0, 0, 0, 0, ...","['MLB135384', '-1', 'MLB135384', '0', '0', '0'...","['MLB-SMARTWATCHES', '-1', 'MLB-SMARTWATCHES',...","[-0.006503703538328409, -0.006754247471690178,...",2
2,1,228737,2019-10-07 19:56:43.678,view,-1.0,"[4, 5, 19, 13, 1, 13, 1, 4, 5, 9, 18, 1, 0, 5,...",search,"['search', '0', '0', '0', '0', '0', '0', '0', ...","[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","[-0.006754247471690178, 0.0, 0.0, 0.0, 0.0, 0....",1
3,1,228737,2019-10-07 20:46:48.382,buy,228737.0,"[14, 1, 14]",view,"['view', 'search', '0', '0', '0', '0', '0', '0...","[228737, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[403016, 402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['MLB264021', '-1', '0', '0', '0', '0', '0', '...","['MLB-MILK_EXTRACTORS', '-1', '0', '0', '0', '...","[-0.006223048083484173, -0.006754247471690178,...",1
4,10000,1910400,2019-10-21 18:22:26.549,buy,1431639.0,"[14, 1, 14]",view,"['view', '0', '0', '0', '0', '0', '0', '0', '0...","[1431639, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[423081, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['MLB278239', '0', '0', '0', '0', '0', '0', '0...","['MLB-FLEA_AND_TICK_TREATMENTS', '0', '0', '0'...","[-0.006220533512532711, 0.0, 0.0, 0.0, 0.0, 0....",1


In [100]:
stats.mode(eval(lst))

ModeResult(mode=array(['a'], dtype='<U1'), count=array([2]))

In [109]:
stats.mode([])[1]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [110]:
from scipy import stats

def count_hist(arr, val='0'):
    arr = np.array(eval(arr))
    arr = arr[arr == val]
    if len(arr) == 0:
        return 0
    
    return int(stats.mode(arr)[1][0])
               

udf_count_hist = udf(count_hist, IntegerType())


In [113]:
_df = df.withColumn('count_search_event_type_history',  udf_count_hist(col("event_type_history"), lit('search')))
_df = _df.withColumn('count__view_event_type_history',  udf_count_hist(col("event_type_history"), lit('view')))

_df = _df.toPandas().head()
_df


,SessionID,ItemID,Timestamp,event_type,last_ItemID,last_event_search,last_event_type,event_type_history,ItemID_history,timestamp_history,category_id_history,domain_id_history,price_history,count_search_event_type_history,count__view_event_type_history
0,0,1615991,2019-10-19 12:27:26.879,view,-1.0,"[18, 5, 12, 15, 7, 9, 15, 0, 19, 13, 1, 18, 20...",search,"['search', 'view', '0', '0', '0', '0', '0', '0...","[-1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[419846, 419845, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['-1', 'MLB135384', '0', '0', '0', '0', '0', '...","['-1', 'MLB-SMARTWATCHES', '0', '0', '0', '0',...","[-0.006754247471690178, -0.006377095822244883,...",1,1
1,0,1748830,2019-10-20 20:29:11.646,buy,1615991.0,"[14, 1, 14]",view,"['view', 'search', 'view', '0', '0', '0', '0',...","[1615991, -1, 1786148, 0, 0, 0, 0, 0, 0, 0, 0,...","[419847, 419846, 419845, 0, 0, 0, 0, 0, 0, 0, ...","['MLB135384', '-1', 'MLB135384', '0', '0', '0'...","['MLB-SMARTWATCHES', '-1', 'MLB-SMARTWATCHES',...","[-0.006503703538328409, -0.006754247471690178,...",1,2
2,1,228737,2019-10-07 19:56:43.678,view,-1.0,"[4, 5, 19, 13, 1, 13, 1, 4, 5, 9, 18, 1, 0, 5,...",search,"['search', '0', '0', '0', '0', '0', '0', '0', ...","[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","['-1', '0', '0', '0', '0', '0', '0', '0', '0',...","[-0.006754247471690178, 0.0, 0.0, 0.0, 0.0, 0....",1,0
3,1,228737,2019-10-07 20:46:48.382,buy,228737.0,"[14, 1, 14]",view,"['view', 'search', '0', '0', '0', '0', '0', '0...","[228737, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[403016, 402465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['MLB264021', '-1', '0', '0', '0', '0', '0', '...","['MLB-MILK_EXTRACTORS', '-1', '0', '0', '0', '...","[-0.006223048083484173, -0.006754247471690178,...",1,1
4,10000,1910400,2019-10-21 18:22:26.549,buy,1431639.0,"[14, 1, 14]",view,"['view', '0', '0', '0', '0', '0', '0', '0', '0...","[1431639, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[423081, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['MLB278239', '0', '0', '0', '0', '0', '0', '0...","['MLB-FLEA_AND_TICK_TREATMENTS', '0', '0', '0'...","[-0.006220533512532711, 0.0, 0.0, 0.0, 0.0, 0....",0,1
